# Classifying Claims - Keras Tokeniser TFIDF + FFNN

In this post we will see if we can build some classifiers to predict a first level patent classification from the claim text.

In particular, here we will look at applying a standard feed forward neural network on a TFIDF matrix.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
# Load data
import os
import pickle

with open("raw_data.pkl", "rb") as f:
    data = pickle.load(f)

In [3]:
data[0]

('\n1. A detector for atrial fibrillation or flutter (AF) comprising: \nan impedance measuring unit comprising a measuring input, to which an atrial electrode line having an electrode for a unipolar measurement of an impedance in an atrium is connected and is implemented to generate an atrial impedance signal, obtained in a unipolar manner, in such a way that an impedance signal for each atrial cycle, comprising an atrial contraction and a following relaxation of said atrium, comprises multiple impedance values detected at different instants within a particular atrial cycle; \nsaid impedance measuring unit comprising a signal input, via which a ventricle signal is to be supplied to said detector, which reflects instants of ventricular contractions in chronological assignment to said impedance signal; \nan analysis unit configured to average multiple sequential impedance signal sections of a unipolar atrial impedance signal, which are each delimited by two sequential ventricular contrac

Let's have a play with the Keras text tokenizer (as per here - https://keras.io/preprocessing/text/#tokenizer).

In [4]:
from keras.preprocessing.text import Tokenizer

docs = [d[0] for d in data]

# create the tokenizer
t = Tokenizer(num_words=10000)
# fit the tokenizer on the documents
t.fit_on_texts(docs)

encoded_claims = t.texts_to_matrix(docs, mode='tfidf')

encoded_claims.shape

(11238, 10000)

Using the texts_to_matrix function we need to apply a feed-forward neural network rather than a RNN, as we have for each claim a set of word counts scaled by document frequency.

This is much faster than my old methods! But hey, I learnt some stuff about tokenisation.  

We can use the "num_words" parameter as passed into the Tokenizer to restrict to the top n words.

In [5]:
Y_class = [d[1] for d in data]

# encode class values as integers
label_e = LabelEncoder()
label_e.fit(Y_class)
encoded_Y = label_e.transform(Y_class)
# convert integers to dummy variables (i.e. one hot encoded)
Y_data = to_categorical(encoded_Y)
print("Our classes are now a matrix of {0}".format(Y_data.shape))
print("Original label: {0}; Converted label: {1}".format(Y_class[0], Y_data[0]))

Our classes are now a matrix of (11238, 8)
Original label: A; Converted label: [ 1.  0.  0.  0.  0.  0.  0.  0.]


In [6]:
# split the data into training (80%) and testing (20%)
(X_train, X_test, Y_train, Y_test) = train_test_split(encoded_claims, Y_data, test_size=0.2)

In [7]:
# clear some memory
del data, docs

In [8]:
input_dim = encoded_claims.shape[1]
print("Our input dimension for our claim representation is {0}".format(input_dim))

Our input dimension for our claim representation is 10000


In [9]:
no_classes = Y_data.shape[1]
print("Our output dimension is {0}".format(no_classes))

Our output dimension is 8


In [ ]:
# create the model
model = Sequential()
model.add(Dropout(0.2, input_shape=(input_dim,)))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(no_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [42]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 8990 samples, validate on 2248 samples
Epoch 1/5
8990/8990 [==============================] - 436s - loss: 0.0635 - acc: 0.9802 - val_loss: 1.5020 - val_acc: 0.6303
Epoch 2/5
8990/8990 [==============================] - 386s - loss: 0.0476 - acc: 0.9853 - val_loss: 1.5905 - val_acc: 0.6312
Epoch 3/5
8990/8990 [==============================] - 671s - loss: 0.0436 - acc: 0.9868 - val_loss: 1.6466 - val_acc: 0.6357
Epoch 4/5
8990/8990 [==============================] - 235s - loss: 0.0431 - acc: 0.9861 - val_loss: 1.6857 - val_acc: 0.6299
Epoch 5/5
8990/8990 [==============================] - 158s - loss: 0.0363 - acc: 0.9890 - val_loss: 1.7460 - val_acc: 0.6268
Accuracy: 62.68%


It is overfitting on our training set. We should try adding dropout.

In [43]:
model.save("2017-10-28 dense_dropout.hd5")

Interesting that this approach doesn't seem to work. Likely because there are limited patterns at the word level to be detected by a CNN.  

This would likely work better at the character level.

In [24]:
#Code for building a confusion matrix

def get_confusion_matrix_one_hot(model_results, truth):
    '''model_results and truth should be for one-hot format, i.e, have >= 2 columns,
    where truth is 0/1, and max along each row of model_results is model result
    '''
    assert model_results.shape == truth.shape
    num_outputs = truth.shape[1]
    confusion_matrix = np.zeros((num_outputs, num_outputs), dtype=np.int32)
    predictions = np.argmax(model_results,axis=1)
    assert len(predictions)==truth.shape[0]

    for actual_class in range(num_outputs):
        idx_examples_this_class = truth[:,actual_class]==1
        prediction_for_this_class = predictions[idx_examples_this_class]
        for predicted_class in range(num_outputs):
            count = np.sum(prediction_for_this_class==predicted_class)
            confusion_matrix[actual_class, predicted_class] = count
    assert np.sum(confusion_matrix)==len(truth)
    assert np.sum(confusion_matrix)==np.sum(truth)
    return confusion_matrix

In [20]:
from collections import Counter
# Let's check how our data is distributed across the classes
class_count = Counter(Y_class)
class_count

Counter({'A': 1777,
         'B': 1449,
         'C': 865,
         'D': 54,
         'E': 269,
         'F': 735,
         'G': 3335,
         'H': 2754})

In [25]:
predict = model.predict(X_test)

In [26]:
cm = get_confusion_matrix_one_hot(predict, Y_test)

In [27]:
cm

array([[212,  37,  30,   0,   3,   9,  30,  15],
       [ 15, 161,   9,   0,   5,  20,  46,  31],
       [ 29,  29,  90,   0,   0,   2,  16,  12],
       [  1,   6,   0,   2,   0,   0,   1,   2],
       [  6,  19,   0,   0,  21,   3,   8,   3],
       [  8,  25,   2,   0,   3,  62,  11,  17],
       [ 15,  28,   6,   0,   1,   9, 512, 129],
       [  9,  22,   7,   0,   2,  14, 123, 370]], dtype=int32)

Could we reduce the classification to a series of binary classifications?

We also really ought to test against a naive Bayes classifier to use as a baseline.  

In [32]:
print(X_train.shape, Y_train.shape)

(8990, 26142) (8990, 8)


In [33]:
Y_train_labelint = np.argmax(Y_train, axis=1)
Y_train_labelint.shape

(8990,)

In [35]:
# For sklearn we need the integer Y labels
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, Y_train_labelint)

In [36]:
predicted = clf.predict(X_test)
np.mean(predicted == np.argmax(Y_test, axis=1))    

0.60186832740213525

The accuracy of the naive Bayes classifier is 60%. 

So I only gain 3% by using a complicated neural network!

In [37]:
predicted[0]

2

In [39]:
# To use the confusion matrix I need to convert predicted into one hot
predicted_one_hot = np.zeros(
    (len(predicted), Y_test.shape[1]),
    dtype='float32')
for i, p in enumerate(predicted):
    predicted_one_hot[i, p] = 1
print(predicted_one_hot[0])

[ 0.  0.  1.  0.  0.  0.  0.  0.]


In [40]:
cm_NB = get_confusion_matrix_one_hot(predicted_one_hot, Y_test)

In [41]:
cm_NB

array([[123,  59,  69,  12,  17,  15,  35,   6],
       [  2, 174,  20,   3,  16,  42,  18,  12],
       [  9,  17, 131,   5,   1,   5,   3,   7],
       [  0,   5,   2,   3,   1,   0,   0,   1],
       [  0,  14,   2,   0,  31,   9,   4,   0],
       [  0,  25,   3,   0,   7,  86,   2,   5],
       [  4,  37,  18,   4,  12,  21, 490, 114],
       [  4,  48,  14,   0,   8,  35, 123, 315]], dtype=int32)

Confusion matrix for NB:
```
       [123,  59,  69,  12,  17,  15,  35,   6],
       [  2, 174,  20,   3,  16,  42,  18,  12],
       [  9,  17, 131,   5,   1,   5,   3,   7],
       [  0,   5,   2,   3,   1,   0,   0,   1],
       [  0,  14,   2,   0,  31,   9,   4,   0],
       [  0,  25,   3,   0,   7,  86,   2,   5],
       [  4,  37,  18,   4,  12,  21, 490, 114],
       [  4,  48,  14,   0,   8,  35, 123, 315]```

In [10]:
# create a deeper model
deep_model = Sequential()
deep_model.add(Dropout(0.2, input_shape=(input_dim,)))
deep_model.add(Dense(2000, activation='relu'))
deep_model.add(Dropout(0.5))
deep_model.add(Dense(500, activation='relu'))
deep_model.add(Dense(no_classes, activation='softmax'))
deep_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              20002000  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 4008      
Total params: 21,006,508
Trainable params: 21,006,508
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
deep_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=64)
# Final evaluation of the model
scores = deep_model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 8990 samples, validate on 2248 samples
Epoch 1/20
8990/8990 [==============================] - 259s - loss: 0.1062 - acc: 0.9640 - val_loss: 1.6931 - val_acc: 0.6366
Epoch 2/20
8990/8990 [==============================] - 265s - loss: 0.1047 - acc: 0.9676 - val_loss: 1.7964 - val_acc: 0.6170
Epoch 3/20
8990/8990 [==============================] - 265s - loss: 0.0828 - acc: 0.9744 - val_loss: 1.7712 - val_acc: 0.6370
Epoch 4/20
8990/8990 [==============================] - 251s - loss: 0.0784 - acc: 0.9765 - val_loss: 1.8022 - val_acc: 0.6321
Epoch 5/20
8990/8990 [==============================] - 238s - loss: 0.0782 - acc: 0.9778 - val_loss: 1.9791 - val_acc: 0.6223
Epoch 6/20
8990/8990 [==============================] - 237s - loss: 0.0688 - acc: 0.9781 - val_loss: 1.9709 - val_acc: 0.6241
Epoch 7/20
8990/8990 [==============================] - 235s - loss: 0.0812 - acc: 0.9778 - val_loss: 1.9101 - val_acc: 0.6388
Epoch 8/20
8990/8990 [==============================] - 262s - 

In [13]:
deep_model.save("2017-10-28 deep_dense_dropout.hd5")